In [ ]:
!pip install datasets
!pip install sentence_transformers
!pip install scikit-learn
!pip install faiss-gpu-cu12

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np
from google.colab import userdata

In [ ]:
ds = load_dataset("ruslanmv/ai-medical-chatbot")

In [ ]:
ds["train"][0]["Description"]

In [ ]:
qa_pairs = [(entry["Patient"], entry["Doctor"]) for entry in ds["train"]]
questions, answers = zip(*qa_pairs)

In [ ]:
# Initialize model with GPU
embed_model = SentenceTransformer("neuml/pubmedbert-base-embeddings", device="cuda")

In [ ]:
# Config

QDRANT_HOST = userdata.get("QDRANT_HOST")
QDRANT_API_KEY = userdata.get("QDRANT_API_KEY")
COLLECTION_NAME = "ruslanmv-ai-medical-chatbot"

In [ ]:
# ---- Initialize Qdrant Client ----
client = QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
)

In [ ]:
# ---- Create Collection ----
if not client.collection_exists(collection_name=COLLECTION_NAME):
    print("Creating collection", COLLECTION_NAME)
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=768,  # Depends on your embedding model, neuml/pubmedbert-base-embeddings is a 768 dimensional dense vector space
            distance=models.Distance.COSINE
        )
    )
else:
    print("Collection already exists")

In [ ]:
# Process embeddings in batches
batch_size = 64  # Larger batch size for GPU
question_embeddings = embed_model.encode(
    questions,
    batch_size=batch_size,
    convert_to_numpy=True,
    show_progress_bar=True,
    device="cuda"
)

# Upload to Qdrant
# client.upload_collection(
#     collection_name=COLLECTION_NAME,
#     vectors=question_embeddings,
#     payloads=[{"answer": answer} for answer in answers],
#     ids=[str(i) for i in range(len(questions))],
#     batch_size=batch_size
# )